# Trabalho Final MD


## Pré-processamento
### Importações necessárias

In [137]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

from sklearn.model_selection import train_test_split



In [138]:
!pip install xlrd


### Leitura dos dados

In [139]:
claims = pd.read_excel("claims-2014.xls")

### Veridicando valores nulos

In [140]:
claims.isnull().sum().sort_values(ascending=False).head(10)

Disposition      3737
Close Amount     3737
Item Category     915
Airline Name      532
Airport Name       66
Airport Code       66
Claim Site         38
Claim Type         36
Incident Date       0
Date Received       0
dtype: int64

### Retirando valores nulos ou atributos desnecessários

In [141]:
claims.drop(['Airport Name', 'Close Amount'], axis=1, inplace= True)

In [142]:
claims.dropna(inplace= True)

In [143]:
claims.isnull().sum().sort_values(ascending=False).head(10)

Disposition      0
Item Category    0
Claim Site       0
Claim Type       0
Airline Name     0
Airport Code     0
Incident Date    0
Date Received    0
Claim Number     0
dtype: int64

In [144]:
claims.head()

,Claim Number,Date Received,Incident Date,Airport Code,Airline Name,Claim Type,Claim Site,Item Category,Disposition
0,2013081805991,2014-01-13,2012-12-21 00:00:00,HPN,USAir,Personal Injury,Checked Baggage,Audio/Video; Jewelry & Watches,Deny
1,2014080215586,2014-07-17,2014-06-30 18:38:00,MCO,Delta Air Lines,Property Damage,Checked Baggage,-I10J13,Deny
2,2014010710583,2014-01-07,2013-12-27 22:00:00,SJU,Jet Blue,Property Damage,Checked Baggage,Food & Drink,Approve in Full
3,2014010910683,2014-01-07,2014-01-02 00:00:00,IAD,UAL,Property Damage,Checked Baggage,Baggage/Cases/Purses,Deny
4,2014011310783,2014-01-09,2014-01-07 00:00:00,SAT,Southwest Airlines,Property Damage,Checked Baggage,Computer & Accessories,Deny


### Dividindo o conjunto em treino e teste e mudando os labels

In [145]:
def label_fix(label):
 
    if label =="Deny":
        return 0
    elif label =="Approve in Full":
        return 1
    elif label=="Settle":
        return 2


In [147]:
claims['Disposition'] = claims['Disposition'].apply(label_fix)

In [148]:
claims.head()

,Claim Number,Date Received,Incident Date,Airport Code,Airline Name,Claim Type,Claim Site,Item Category,Disposition
0,2013081805991,2014-01-13,2012-12-21 00:00:00,HPN,USAir,Personal Injury,Checked Baggage,Audio/Video; Jewelry & Watches,0
1,2014080215586,2014-07-17,2014-06-30 18:38:00,MCO,Delta Air Lines,Property Damage,Checked Baggage,-I10J13,0
2,2014010710583,2014-01-07,2013-12-27 22:00:00,SJU,Jet Blue,Property Damage,Checked Baggage,Food & Drink,1
3,2014010910683,2014-01-07,2014-01-02 00:00:00,IAD,UAL,Property Damage,Checked Baggage,Baggage/Cases/Purses,0
4,2014011310783,2014-01-09,2014-01-07 00:00:00,SAT,Southwest Airlines,Property Damage,Checked Baggage,Computer & Accessories,0


In [153]:
X_data = claims[['Claim Number','Date Received','Incident Date','Airport Code','Airline Name', 'Claim Type', 'Claim Site', 'Item Category']]
y_labels = claims['Disposition']

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y_labels,test_size = 0.3)

In [157]:
import tensorflow as tf

In [161]:
claims.columns

Index(['Claim Number', 'Date Received', 'Incident Date', 'Airport Code',
       'Airline Name', 'Claim Type', 'Claim Site', 'Item Category',
       'Disposition'],
      dtype='object')

## Create the Feature  Columns for tf.esitmator

In [170]:
Airport_Code = tf.feature_column.categorical_column_with_hash_bucket("Airport Code", hash_bucket_size=1000)
Claim_Type = tf.feature_column.categorical_column_with_hash_bucket("Claim Type", hash_bucket_size=1000)
Claim_Site = tf.feature_column.categorical_column_with_hash_bucket("Claim Site", hash_bucket_size=1000)
Item_Category = tf.feature_column.categorical_column_with_hash_bucket("Item Category", hash_bucket_size=10000)
Airline_Name = tf.feature_column.categorical_column_with_hash_bucket("Airline Name", hash_bucket_size= 1000)

Claim_Number = tf.feature_column.numeric_column("Claim Number")
Date_Received = tf.feature_column.categorical_column_with_hash_bucket("Date Received", hash_bucket_size=100000)
Incident_Date = tf.feature_column.categorical_column_with_hash_bucket("Incident Date", hash_bucket_size= 100000)

In [171]:
feat_cols = [Claim_Number,Date_Received,Incident_Date,Airport_Code,
             Airline_Name,Claim_Type,Claim_Site,Item_Category]

## Input Function

In [172]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [173]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1ph4ssv2', '_save_summary_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_save_checkpoints_steps': None}


In [174]:
model.train(input_fn=input_func,steps=5000)

TypeError: Cannot convert value dtype('<M8[ns]') to a TensorFlow DType.